In [1]:
import pickle
import os
import time
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from numpy import linalg as LA
import numpy as np
import math

with open("17_mnist.pkl", "br") as fh:
    data = pickle.load(fh)

train_imgs = data[0]
test_imgs = data[1]
train_labels = data[2]
test_labels = data[3]



In [2]:
import numpy as np
for i in range(len(train_labels)):
    if train_labels[i]==7:
        train_labels[i]=np.array([0.])

In [3]:
train_x = torch.FloatTensor(train_imgs)
train_y = torch.FloatTensor(train_labels)
#train_y = train_y.squeeze(1)

In [4]:
class LinearDataset(Dataset):
    def __init__(self, X, y):
        assert X.size()[0] == y.size()[0]
        self.X = X
        self.y = y
    
    def __len__(self):
        return self.X.size()[0]
    
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]

In [13]:
device='cuda'

class LR(nn.Module):
    def __init__(self, input_size=784, num_class=1):
        super(LR, self).__init__()
        self.fc1 = nn.Linear(input_size, num_class)
        self.act=nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        return (x)

In [14]:
train_loader = torch.utils.data.DataLoader(LinearDataset(train_x, train_y), batch_size=len(train_imgs))

In [15]:
model = LR().to(device).double()

In [95]:
state_dict = model.state_dict()
print(state_dict['fc1.weight'].size())
state_dict['fc1.weight'] = torch.ones(1,784)
state_dict['fc1.bias'] = torch.Tensor([-20])
model.load_state_dict(state_dict)

torch.Size([1, 784])


<All keys matched successfully>

In [97]:
def autograd(outputs, inputs, create_graph=False):
    """Compute gradient of outputs w.r.t. inputs, assuming outputs is a scalar."""
    #inputs = tuple(inputs)
    grads = torch.autograd.grad(outputs, inputs, create_graph=create_graph, allow_unused=True)
    return [xx if xx is not None else yy.new_zeros(yy.size()) for xx, yy in zip(grads, inputs)]

from scipy.special import lambertw
import math
lambert_w = lambertw(1/math.e)



def tau(epoch,lr):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device).double(), target.to(device).double()
        # initialize f function
        criterion = nn.BCELoss()
        
        # calculate gradient of w on clean sample
        output_c = model(data.view(data.size(0), -1))
        #output_c = model(data)
        loss_c = criterion(output_c,target)
        # wrt to w here
        grad_c= autograd(loss_c,tuple(model.parameters()),create_graph=True)
        g1 = grad_c[0]
        torch.save(g1.detach(), 'gmu_lr.pt')
        
        g1 = torch.flatten(g1)
        
        
        
        
        w_p = model.fc1.weight
        
        w_p = torch.flatten(w_p)
        g_mu_dot_w = np.dot(g1.cpu().detach().numpy().squeeze(),w_p.cpu().detach().numpy().squeeze())
        print(g_mu_dot_w)
        
        print('the necessary size of epsilon_d:{}'.format(g_mu_dot_w/lambert_w))
        break
            

        
        
print("==> start gradient canceling attack with given target parameters")
print("==> model will be saved in poisoned_models")
epochs=1
lr=1
for epoch in range(epochs):
    tau(epoch,lr)
    break
    

==> start gradient canceling attack with given target parameters
==> model will be saved in poisoned_models
tensor([[0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
        [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
        [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
        ...,
        [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
        [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
        [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100]],
       device='cuda:0', dtype=torch.float64)
0.06314240582474075
the necessary size of epsilon_d:(0.2267520496457524+0j)


In [74]:
optimizer = optim.Adadelta(model.parameters(), lr=0.1)
# define epsilon for one point for now
epsilon = 0.01



# load gmu to initialize the poisoned samples
gmu = torch.load('gmu_lr.pt')
print(gmu.size())

loss_all = []
def attack(epoch,lr):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device).double(), target.to(device).double()
        data.requires_grad=True
        if epoch==0:
            # initialize poisoned data with 1/epsilon g(mu)
            #data_p = Variable(data[:epsilon])
            data_p = Variable((gmu))
            target_p = Variable(torch.ones([1,1])).to(device).double()
            torch.save(target_p,'target_p_{}.pt'.format(epsilon))
        else:
            data_p = torch.load('data_p_{}.pt'.format(epsilon))
            target_p = torch.load('target_p_{}.pt'.format(epsilon))
        data_p.requires_grad=True
        # initialize f function with average
        criterion = nn.BCELoss()
        
        # calculate gradient of w on clean sample
        output_c = model(data.view(data.size(0), -1))
        loss_c = criterion(output_c,target)
        # wrt to w here
        grad_c= autograd(loss_c,tuple(model.parameters()),create_graph=True)
        g1 = grad_c[0]
        
        
        # calculate gradient of w on poisoned sample
        output_p = model(data_p.view(data_p.size(0), -1))

        loss_p = criterion(output_p,target_p)
        grad_p= autograd(loss_p,tuple(model.parameters()),create_graph=True)
        g2 = grad_p[0]
        # calculate the true loss: |g_c + g_p|_{inf}
        
        grad_sum = g1+(epsilon*g2)

        
        loss = torch.norm(grad_sum,2).square()
        loss_all.append(loss.detach().cpu().numpy())
            
        update = autograd(loss,data_p,create_graph=True)
        #print(update)
        
        data_t = data_p - lr * update[0]

        torch.save(data_t, 'data_p_{}.pt'.format(epsilon))
        
        
        print("epoch:{},loss:{},lr:{}".format(epoch, loss,lr))

        
        
print("==> start gradient canceling attack with given target parameters")
print("==> model will be saved in poisoned_models")
epochs=2000
lr= 0.1
for epoch in range(epochs):
    attack(epoch,lr)
    

torch.Size([1, 784])
==> start gradient canceling attack with given target parameters
==> model will be saved in poisoned_models
epoch:0,loss:5.790360160597739e-33,lr:0.1
epoch:1,loss:5.7903434080583965e-33,lr:0.1
epoch:2,loss:5.79032665556752e-33,lr:0.1
epoch:3,loss:5.790309903125115e-33,lr:0.1
epoch:4,loss:5.790293150731175e-33,lr:0.1
epoch:5,loss:5.7902763983857034e-33,lr:0.1
epoch:6,loss:5.7902596460887e-33,lr:0.1
epoch:7,loss:5.7902428938401625e-33,lr:0.1
epoch:8,loss:5.790226141640095e-33,lr:0.1
epoch:9,loss:5.790209389488492e-33,lr:0.1
epoch:10,loss:5.790192637385356e-33,lr:0.1
epoch:11,loss:5.790175885330689e-33,lr:0.1
epoch:12,loss:5.790159133324487e-33,lr:0.1
epoch:13,loss:5.790142381366751e-33,lr:0.1
epoch:14,loss:5.7901256294574835e-33,lr:0.1
epoch:15,loss:5.79010887759668e-33,lr:0.1
epoch:16,loss:5.7900921257843426e-33,lr:0.1
epoch:17,loss:5.790075374020472e-33,lr:0.1
epoch:18,loss:5.790058622305069e-33,lr:0.1
epoch:19,loss:5.790041870638129e-33,lr:0.1
epoch:20,loss:5.7900

KeyboardInterrupt: 